This is a project I did while I was studying machine learning at the University of Melbourne in 2019. Yes, we can use scikit-learn to build a much handier and more comprehensive Naive Bayes classifier -  This was just an exercise with the purpose to <i> understand </i> what's behind Naive Bayes models.

# Naive Bayes Clssifier - built from scratch
#### This is a Naive Bayes classifier built without the help of any packages (except 'math'), even pandas.  It reads a csv file whose last column of each row indicats the class of the instance, trains the model and predicts the class of instances. It also outputs accuracy and error rate of the classifier and information gain of each attribute in the dataset. In the end, this script investigated the impact of missing values on a Naive Bayes model.

In [1]:
import math

In [2]:
# I used the whole dataset as both training and testing dataset for all datasets in this project.
# modify the following according to the dataset of interest
f = open('hepatitis.csv','r')

## Preprocessing

In [3]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(file):
    dataset=[]
    for line in file.readlines():
        dataset.append(line.strip().split(','))
    return dataset

dataset = preprocess(f)
#print(dataset)

In [4]:
# generate a list of dictionaries in the format of {attribute value : counts}
def attribute_list(dataset):
    '''for each attribute, count the number of instances that have a certain value for that attribtue'''
    list_attr=[]
    for x in range(len(dataset[0])-1):
        list_attr.append({})

    for i in dataset:
        for c in range(len(i)-1):
            if i[c] not in list_attr[c].keys():
                list_attr[c][i[c]] = 1
            else:
                list_attr[c][i[c]] += 1
    return list_attr

## Training the Model

In [5]:
# This function should build a supervised NB model
def train(dataset):
    '''The posteriors model is a list of nested dictionaries, where each item in the list represents 
    the counts associated with one attribute.
    The attributes are ordered as in the dataset.'''
    model=[{} for i in range(len(dataset[0])-1)] # a list of dictionaries as framework for counting
    # counting priors and posteriors
    count_instances = 0
    class_dict = {}

    for instance in dataset:
        # priors
        count_instances+=1
        if instance[-1] not in class_dict.keys():
            class_dict[instance[-1]] = 1
        else:
            class_dict[instance[-1]] += 1
        # posteriors
        for index in range(len(instance)-1):
            if instance[-1] not in model[index].keys():
                model[index].update({instance[-1]:{}})
            if instance[index] not in model[index][instance[-1]].keys():
                model[index][instance[-1]].update({instance[index]:1})
            else:                
                model[index][instance[-1]][instance[index]]+=1
    return {'count of instances': count_instances, 'priors': class_dict, 'posteriors': model}
model = train(dataset)
###### If an IndexError occurs, restart the file and re-run this function (because the dataset was trained more than once)

In [6]:
print(model)

{'count of instances': 155, 'priors': {'LIVE': 123, 'DIE': 32}, 'posteriors': [{'LIVE': {'female': 16, 'male': 107}, 'DIE': {'male': 32}}, {'LIVE': {'no': 56, 'yes': 66, '?': 1}, 'DIE': {'no': 20, 'yes': 12}}, {'LIVE': {'yes': 101, 'no': 22}, 'DIE': {'yes': 30, 'no': 2}}, {'LIVE': {'yes': 52, 'no': 70, '?': 1}, 'DIE': {'no': 30, 'yes': 2}}, {'LIVE': {'yes': 84, 'no': 38, '?': 1}, 'DIE': {'yes': 9, 'no': 23}}, {'LIVE': {'yes': 100, 'no': 22, '?': 1}, 'DIE': {'no': 10, 'yes': 22}}, {'LIVE': {'no': 22, 'yes': 96, '?': 5}, 'DIE': {'yes': 24, '?': 5, 'no': 3}}, {'LIVE': {'yes': 70, 'no': 47, '?': 6}, 'DIE': {'yes': 14, 'no': 13, '?': 5}}, {'LIVE': {'yes': 101, 'no': 18, '?': 4}, 'DIE': {'no': 12, 'yes': 19, '?': 1}}, {'LIVE': {'yes': 90, 'no': 29, '?': 4}, 'DIE': {'no': 22, 'yes': 9, '?': 1}}, {'LIVE': {'yes': 113, 'no': 6, '?': 4}, 'DIE': {'yes': 17, 'no': 14, '?': 1}}, {'LIVE': {'yes': 112, '?': 4, 'no': 7}, 'DIE': {'yes': 20, 'no': 11, '?': 1}}, {'LIVE': {'no': 78, 'yes': 45}, 'DIE': {'n

## Predicting

In [7]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instance):
    '''The instance is supposed to have the same format as instances in the preprocessed dataset 
    (a list of attribute values, where the attributes are ordered as in the dataset.).'''
    prob_classes={}

    for elem in model['priors'].keys():
        prob_classes.update({elem : (model['priors'][elem]/model['count of instances'])})

    for index in range(len(instance)-1):
        for key in prob_classes.keys():
        # if there is no combination of attribute value and class of the instance, add that combination into the model with a count of 0
            if instance[index] not in model['posteriors'][index][key].keys():
                model['posteriors'][index][key].update({instance[index] : 0})

    # create a list of dictionaries: {class : probability(attribute = value | class)}
    prob_cond=[]
    epsilon = 0.00001 # epsilon smoothing, this value may be edited
    for index in range(len(instance)-1):
        prob_cond.append({})
        for key in prob_classes.keys():
            # smoothing
            conditional_prob = (model['posteriors'][index][key][instance[index]] / model['priors'][key])
            if conditional_prob == 0:
                conditional_prob = epsilon
            prob_cond[index].update({ key : conditional_prob})

    #calculating the probability that the instance belongs to each class
    predict_class_prob = {key: 1 for key in prob_classes.keys()}
    for index in range(len(prob_cond)-1):
        for key in prob_classes.keys():
            predict_class_prob[key] *= prob_cond[index][key]

    for key in predict_class_prob.keys():
        if predict_class_prob[key] == max(predict_class_prob.values()):
            predicted_class = key
    
    return predicted_class

In [8]:
# a demonstration of the predict() function
instance = dataset[0]
print(instance)
print('The predicted class of this instance is', predict(model, instance))

['female', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'LIVE']
The predicted class of this instance is LIVE


## Evaluating the Model

In [9]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(model, dataset):
    #create a nested list where each item is a pair of [actual class vs. predicted class] for each instance
    predictions = []

    for instance in dataset:
        true_class = instance[-1]
        predicted_class = predict(model, instance)
        predictions.append([true_class,predicted_class])
    #print(predictions)

    # accuracy
    count_true = 0
    count_total = 0
    for elem in predictions:
        count_total +=1
        if elem[0] == elem[1]:
            count_true +=1
    accuracy = count_true/count_total
    error_rate = 1 - accuracy
    
    
    print('The accuracy for this model and dataset is', round(accuracy,4))
    print('The error rate for this model and dataset is', round(error_rate,4))

evaluate(model, dataset)

The accuracy for this model and dataset is 0.8258
The error rate for this model and dataset is 0.1742


In [10]:
def entropy(list_of_prob): 
    '''calculates entropy given a list of probabilities'''
    e = 0
    for elem in list_of_prob:
        e += (elem * math.log(elem,2))
    entropy = -e
    return entropy

## Calculating Information Gain

In [11]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(model):
    '''This function returns a list of infomation gain for each attribute. 
    The attributes are ordered as in the dataset.'''
    # entropy at top level
    list_of_count = model['priors'].values()
    list_of_prob = []
    for elem in list_of_count:
        prob = elem / sum(list_of_count)
        list_of_prob.append(prob)
    top_level_entropy = entropy(list_of_prob)
    #print(top_level_entropy)

    #list of prob for one attribute value
    #framework
    attr_list_prob = []
    for index in range(len(attribute_list(dataset))):
        attr_list_prob.append({})
        for key in attribute_list(dataset)[index].keys():
            attr_list_prob[index].update({key:[]})

    for index in range(len(model['posteriors'])):
        for cls in model['posteriors'][index].keys():
            for key in model['posteriors'][index][cls].keys():
                # number of instance of one class that has this value            
                num_ins = model['posteriors'][index][cls][key] 
                #total number of instances that have this attribtue value
                ttl_ins = attribute_list(dataset)[index][key]
                prob = num_ins / ttl_ins
                list_of_prob = attr_list_prob[index][key]
                list_of_prob.append(prob)
    #print(attr_list_prob)

    # calculating entropy for each attribute value
    list_value_ent = []
    for index in range(len(attr_list_prob)):
        list_value_ent.append({})
        for key in attr_list_prob[index].keys():
            if 0 in attr_list_prob[index][key]:
                entropy_of_value = 0
            else:
                entropy_of_value = entropy(attr_list_prob[index][key])
            list_value_ent[index].update({key : entropy_of_value})
    #print(list_value_ent)

    list_attr_MI = []
    list_attr_IG = []
    for index in range(len(list_value_ent)):
        #list_attr_MI.append({})
        MI = 0
        for key in list_value_ent[index].keys():
            #total number of instances that have this attribtue value:
            value_ins = attribute_list(dataset)[index][key]
            total_ins = sum(attribute_list(dataset)[index].values())
            info = (value_ins/total_ins) * list_value_ent[index][key]
            MI += info
            IG = round(top_level_entropy - MI, 4)
        list_attr_MI.append(MI)
        list_attr_IG.append(IG)
    #print(list_attr_MI)
    return top_level_entropy, list_attr_IG

print(info_gain(model))

(0.7346451526501956, [0.0366, 0.0153, 0.0145, 0.0865, 0.0832, 0.0138, 0.0258, 0.0199, 0.0355, 0.1061, 0.1283, 0.0768, 0.0849])


## A run-through for a list of sample datasets

In [12]:
############################## printing results for all datasets ##############################
datasets = ['hepatitis.csv', 'primary-tumor.csv', 'breast-cancer.csv', 'cmc.csv', 'car.csv', 'anneal.csv', 'hypothyroid.csv', 'mushroom.csv', 'nursery.csv']
for data in datasets:
    print(data)
    f = open(data,'r')
    dataset = preprocess(f)
    model = train(dataset)
    evaluate(model, dataset)
    top_level_entropy = info_gain(model)[0]
    list_of_IG = info_gain(model)[1]
    print('the top level entropy for this dataset is', round(top_level_entropy,4))
    print('Information Gain for attributes:', list_of_IG)
    avg = round(sum(list_of_IG)/len(list_of_IG), 4)
    print('the arithmetic average of attribute IG\'s for this dataset:', avg)
    print('*'*50)

hepatitis.csv
The accuracy for this model and dataset is 0.8258
The error rate for this model and dataset is 0.1742
the top level entropy for this dataset is 0.7346
Information Gain for attributes: [0.0366, 0.0153, 0.0145, 0.0865, 0.0832, 0.0138, 0.0258, 0.0199, 0.0355, 0.1061, 0.1283, 0.0768, 0.0849]
the arithmetic average of attribute IG's for this dataset: 0.0559
**************************************************
primary-tumor.csv
The accuracy for this model and dataset is 0.5221
The error rate for this model and dataset is 0.4779
the top level entropy for this dataset is 3.6437
Information Gain for attributes: [3.6437, 3.6437, 3.6437, 3.6437, 3.6437, 0.0584, 3.6437, 3.6437, 3.6437, 3.6437, 0.1774, 0.1914, 3.6437, 3.6437, 0.407, 0.8739, 3.6437]
the arithmetic average of attribute IG's for this dataset: 2.6725
**************************************************
breast-cancer.csv
The accuracy for this model and dataset is 0.7203
The error rate for this model and dataset is 0.2797
the t

## Investigation around Missing Values

In [13]:
# create new datasets for instances with and without missing values
def missing_sets(dataset):
    missing_set = []
    no_miss_set = []
    for instance in dataset:
        for value in instance:
            if value == '?':
                missing_set.append(instance)
        if '?' not in instance:
            no_miss_set.append(instance)
    return missing_set, no_miss_set

In [14]:
# datasets with missing values
datasets = ['hepatitis.csv', 'primary-tumor.csv', 'breast-cancer.csv', 'hypothyroid.csv', 'mushroom.csv']

print('performance of data with MISSING VALUES:')
for data in datasets:
    print(data)
    f = open(data,'r')
    dataset = preprocess(f)
    ds = missing_sets(dataset)[0]
    model = train(ds)
    evaluate(model, ds)
    print('*'*50)

print('-'*65)
print('performance of data with NO MISSING VALUES:')
for data in datasets:
    print(data)
    f = open(data,'r')
    dataset = preprocess(f)
    ds = missing_sets(dataset)[1]
    model = train(ds)
    evaluate(model, ds)
    print('*'*50)


performance of data with MISSING VALUES:
hepatitis.csv
The accuracy for this model and dataset is 1.0
The error rate for this model and dataset is 0.0
**************************************************
primary-tumor.csv
The accuracy for this model and dataset is 0.5867
The error rate for this model and dataset is 0.4133
**************************************************
breast-cancer.csv
The accuracy for this model and dataset is 1.0
The error rate for this model and dataset is 0.0
**************************************************
hypothyroid.csv
The accuracy for this model and dataset is 0.3288
The error rate for this model and dataset is 0.6712
**************************************************
mushroom.csv
The accuracy for this model and dataset is 1.0
The error rate for this model and dataset is 0.0
**************************************************
-----------------------------------------------------------------
performance of data with NO MISSING VALUES:
hepatitis.csv
The accur

In [ ]:
# generate a list of modes
def mode_list(dataset):
    mode_list=[]
    list_attr = attribute_list(dataset)
    for dicts in list_attr:
        mode = list(dicts.keys())[0]
        for key in dicts.keys():
            if dicts[key] > dicts[mode] and key != '?': #### delete " and key != '?' " if keeping missing values
                mode = key
        mode_list.append(mode)
    return mode_list

# impute missing values by the attibute average (i.e. mode, in the case of categorical data)
def mean_imputation(dataset):
    for i in dataset:
        for n in range(len(i)-1):
            if i[n] == '?':
                i[n] = mode_list(dataset)[n]
    return dataset


In [ ]:
print('new performance of dataset with missing values after mean imputation')

for data in datasets:
    print(data)
    f = open(data,'r')
    dataset = preprocess(f)
    imputed_dataset = mean_imputation(missing_sets(dataset)[0])
    model = train(imputed_dataset)
    evaluate(model, imputed_dataset)
    print('*'*50)

new performance of dataset with missing values after mean imputation
hepatitis.csv
The accuracy for this model and dataset is 0.8
The error rate for this model and dataset is 0.2
**************************************************
primary-tumor.csv
The accuracy for this model and dataset is 0.5156
The error rate for this model and dataset is 0.4844
**************************************************
breast-cancer.csv
The accuracy for this model and dataset is 1.0
The error rate for this model and dataset is 0.0
**************************************************
hypothyroid.csv
The accuracy for this model and dataset is 0.3288
The error rate for this model and dataset is 0.6712
**************************************************
mushroom.csv


## Answering a few Questions

These question are answered on the basis of epsilon smoothing with epsilon = 0.00001.


###### Q1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.


The accuracy metric of the classifier varies greatly with respect to the dataset, for example: the accuracy of the ‘mushroom’ dataset is as high as 0.9926 and 0.9889 for the ‘anneal’ dataset, whereas that for the hypothyroid dataset is only 0.356. Such variation can be partly explained by the information gain criterion. For datasets ‘mushroom’, ‘anneal’, ‘hepatitis’, ‘breast cancer’, ‘cmc’ and ‘hypothyroid’ (majority of the datasets), the accuracy generally decreases as the IG (arithmetically averaged from all attributes) decreases. However, the datasets ‘car’, ‘primary tumor’, ‘nursery’, and potentially ‘anneal’ are exceptions to this rule. One possible explanation is that these datasets have relatively high top-level entropy for the classes (however, this may not apply for ‘cmc’,) than other datasets. By observation, the measure MI does not seem to vary greatly among datasets, as IG = top entropy – MI, this higher top-level entropy would induce a higher IG for attributes. 

Also, these four datasets have both relatively large number of classes and attributes (both at least 4), which explains the high entropy values. This may be the reason why ‘cmc’ has high top-level entropy but an IG that is proportional to accuracy (because of only 3 classes). Also, although the dataset ‘mushroom’ has 22 classes, it has only 2 attributes, so the generalisation (dataset with both relatively large number of classes and attributes will have a higher top-level entropy and an IG that is not proportional to accuracy) could still hold.

In addition, special attention may be paid to the datasets ‘primary tumor’ and ‘anneal’, where the actual number of classes is more than what is indicated in the dataset. This could contribute to the high entropy since the model would be less predicative in the case of “missing classes”.


###### Q6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

For datasets ‘primary tumor’, ‘hepatitis’, and ‘breast cancer’, the accuracies of predictions on instances with and without missing values seem to differ, although the degree of difference is arguably acceptable. This could be the evidence that Naive Bayes performs ‘elegantly’ in the presence of missing values. For example, for ‘primary tumor’, the accuracy of predictions on data with missing values is 0.5687, while that for data where all values are present is 0.7045. This should be because that these datasets have small sample sizes. On contrary, for the datasets ‘mushroom’ and ‘hypothyroid’, the difference of accuracies between two sets of data is rather small (e.g. for ‘mushroom’, 0.997 vs. 1), since they have relatively large sample sizes.

By observation, the number of values missing does not seem to matter. For the three datasets mentioned above, they happen to have large, moderate and small number of values missing, i.e. three different degrees of values missing. Still, they all differ in the performance of prediction on data with or without missing values.

Counterintuitively, an imputation strategy (here I used attribute mode imputation) does not seem to improve the performances, and sometimes worsens them. This can be observed from datasets ‘primary tumor’ and ‘hepatitis’(from 0.5687 to 0.5156 and from 1 to 0.8). Both of the datasets have small sample sizes and a relatively large number of attributes. For other datasets, the accuracy after imputation stays the same as before imputation. 